In [1]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
import IPython.display as display
import pprint as ppq


# Global variables to drive stuff - not good practice but this is my code dangit
DEBUG: int = 1
RANDOMSEED: int = 10024


# Node tree class, used for the BTree
class BTreeNode:
    def __init__(self):
        self.right: BTreeNode = None
        self.left: BTreeNode = None
        self.data = None
        self.pivot: int = None
        self.moreOf: int = None

    def GenerateConfusionMatrix(self, isLeft):
        # do the thing
        confusion = dict()

        t = self.__FindPosNeg(0)
        print("final confusion tuple: " + str(t))
        confusion["Actual Positive"] = [t[0], t[3]]
        confusion["Actual Negative"] = [t[1], t[2]]

        confusionmatrix = pandas.DataFrame.from_dict(confusion, "index")
        confusionmatrix.columns = ["Predicted Positive", "Predicted Negative"]

        return confusionmatrix

    # walk the list and get all values
    def __FindPosNeg(self, isRight):  # return a tuple [TP, FP, TN, FN]
        returnTuple = [0, 0, 0, 0]
        print("\nFindPosNeg\n")
        if self.left is None and self.right is None:  # get the actual values
            x = self.__GetConfusionForNode(isRight)
            print("Confusion for node: isRight: " + str(isRight) + " return: " + str(x))
            return x
        else:
            l = self.left.__FindPosNeg(0)
            r = self.right.__FindPosNeg(1)
            print("left: " + str(l) + " right: " + str(r) + "\n")
            returnTuple[0] = returnTuple[0] + l[0] + r[0]
            returnTuple[1] = returnTuple[1] + l[1] + r[1]
            returnTuple[2] = returnTuple[2] + l[2] + r[2]
            returnTuple[3] = returnTuple[3] + l[3] + r[3]
            print("final tuple calculation for this block: " + str(returnTuple))
            return returnTuple

    def __GetConfusionForNode(self, isRight):
        mReadableData = self.data

        t = isRight

        countTruePos = 0
        countTrueNeg = 0
        countFalsePos = 0
        countFalseNeg = 0
        for i in range(0, len(mReadableData.index), 1):
            c = mReadableData.iloc[i, 0]

            print("iteration " + str(i) + " isright: " + str(isRight) + " true value: " + str(c))

            if c == 1:
                if c == t:
                    countTruePos = countTruePos + 1
                else:
                    countFalseNeg = countFalseNeg + 1
            else:
                if c != t:
                    countFalsePos = countFalsePos + 1
                else:
                    countTrueNeg = countTrueNeg + 1
        return [countTruePos, countFalsePos, countTrueNeg, countFalseNeg]

    def MakeConfusionWithTree(self, mReadableData):  # return a tuple [TP, FP, TN, FN]
        returnTuple = [0, 0, 0, 0]

        t = -1
        for i in range(0, len(mReadableData.index), 1):
            c = mReadableData.iloc[i, 0]
            root = self
            while root.left is not None and root.right is not None:

                pivot = mReadableData.iloc[i, root.pivot]

                if pivot == 1:
                    root = root.right
                    t = 1
                else:
                    root = root.left
                    t = 0

            if c == 1:
                if c == t:
                    returnTuple[0] = returnTuple[0] + 1
                else:
                    returnTuple[3] = returnTuple[3] + 1
            else:
                if c != t:
                    returnTuple[1] = returnTuple[1] + 1
                else:
                    returnTuple[2] = returnTuple[2] + 1

        return returnTuple

    def __GetMoreOf(self):  # 1: positive, 0: negative
        p = 0
        np = 0
        for i in range(0, len(self.data.index), 1):
            c = mReadableData.iloc[i, 0]

            if (c == 1):
                p = p + 1
            else:
                np = np + 1

        if p >= np:
            return 1
        else:
            return 0

    def PopulateMoreRoots(self):
        if self.left is not None and self.right is not None:
            self.left.PopulateMoreRoots()
            self.right.PopulateMoreRoots()
        else:
            self.moreOf = self.__GetMoreOf()

    def MakePhiConfusion(self, mReadableData):
        returnTuple = [0, 0, 0, 0]

        t = -1
        for i in range(0, len(mReadableData.index), 1):
            c = mReadableData.iloc[i, 0]
            root = self
            while root.left is not None and root.right is not None:

                pivot = mReadableData.iloc[i, root.pivot]

                if pivot == 1:
                    root = root.right
                else:
                    root = root.left

            t = root.moreOf
            if c == 1:
                if c == t:
                    returnTuple[0] = returnTuple[0] + 1
                else:
                    returnTuple[3] = returnTuple[3] + 1
            else:
                if c != t:
                    returnTuple[1] = returnTuple[1] + 1
                else:
                    returnTuple[2] = returnTuple[2] + 1

        return returnTuple


# Set up all our stuff
def initialize():
    plt.rcParams["figure.figsize"] = [100, 50]
    plt.rcParams.update({'font.size': 15})

    data = pandas.read_csv("mutations.csv")

    print("csv loaded...")

    pandas.set_option('display.max_columns', 30)

    return data


# Turn data into a more machine readable format
def MakeMachineReadable(data):
    # Generate our datatable for confusion matrix

    d = data
    # first off we're going to generate a more machine readable dataset by transforming the first column into 1s (for cancer) and 0s (noncancer)
    for i in range(0, 230, 1):
        dataAt = data.iloc[i, 0]
        if dataAt[0] == 'N':
            d.iloc[i, 0] = 0
        else:
            d.iloc[i, 0] = 1
    display(d)
    return d
    # d now contains the machine-readable data so we'll now use that


def findMaxPhi(mReadableData):
    maxDict = list()

    numSamples = len(mReadableData.index)

    if (DEBUG):
        print("numsamples: " + str(numSamples))

    for j in range(1, len(mReadableData.columns), 1):
        sam = mReadableData.columns[j]
        # right is positive

        NumSamplesR = 0
        NumSamplesL = 0
        PosSamplesL = 0
        NegSamplesL = 0
        PosSamplesR = 0
        NegSamplesR = 0

        for i in range(0, len(mReadableData.index), 1):
            if (mReadableData.iloc[i, j] == 1):  # right

                NumSamplesR = NumSamplesR + 1

                if (mReadableData.iloc[i, 0] == 1):
                    PosSamplesR = PosSamplesR + 1
                else:
                    NegSamplesR = NegSamplesR + 1
            else:  # left

                NumSamplesL = NumSamplesL + 1

                if (mReadableData.iloc[i, 0] == 1):
                    PosSamplesL = PosSamplesL + 1
                else:
                    NegSamplesL = NegSamplesL + 1

            PL = (NumSamplesL / numSamples)
            PR = (NumSamplesR / numSamples)
            PPosL = 0 if NumSamplesL == 0 else PosSamplesL / NumSamplesL
            PNegL = 0 if NumSamplesL == 0 else NegSamplesL / NumSamplesL
            PPosR = 0 if NumSamplesR == 0 else PosSamplesR / NumSamplesR
            PNegR = 0 if NumSamplesR == 0 else NegSamplesR / NumSamplesR
            Q = np.abs(PPosL - PPosR) + np.abs(PNegL - PNegR)
            Phi = (2 * PL * PR) * Q
            maxDict.append(
                [j, sam, NumSamplesL, NumSamplesR, PosSamplesL, NegSamplesL, PL, PR, PNegL, PPosL, PPosR, PNegR,
                 (2 * PL * PR), Q, Phi])

    maxDict.sort(key=lambda x: x[14])
    maxDict.reverse()

    return maxDict


# Find the max TPSubFP of a machine readable dataframe
def FindMaxTPSubFP(mReadableData):
    trueSub = 0
    trueIndex = 0
    tpSubList = list()

    for j in range(1, len(mReadableData.columns), 1):

        countFalsePos = 0
        countTruePos = 0
        countFalseNeg = 0
        countTrueNeg = 0

        for i in range(0, len(mReadableData.index), 1):
            c = mReadableData.iloc[i, 0]
            t = mReadableData.iloc[i, j]

            if c == 1:
                if c == t:
                    countTruePos = countTruePos + 1
                else:
                    countFalseNeg = countFalseNeg + 1
            else:
                if c != t:
                    countFalsePos = countFalsePos + 1
                else:
                    countTrueNeg = countTrueNeg + 1

        tempSub = countTruePos - countFalsePos
        tpSubList.append([j, tempSub])

    tpSubList.sort(key=lambda x: x[1])
    tpSubList.reverse()

    print("Found MaxTPSubFP: " + str(tpSubList[0]) + " or " + str(tpSubList[1]))

    return tpSubList

def __FindEntropy(mReadableData):
    numSamples = len(mReadableData.index)
    NumPosSamples = 0
    NumNegSamples = 0
    for i in range(0, len(mReadableData.index), 1):
        if (mReadableData.iloc[i, 0] == 1):  # right
            NumPosSamples = NumPosSamples + 1
        else:  # left
            NumNegSamples = NumNegSamples + 1

    ProbPos = -1
    ProbNeg = -1

    if numSamples == 0:
        ProbPos = 0
        ProbNeg = 0

    else:
        ProbPos = NumPosSamples / numSamples
        ProbNeg = NumNegSamples / numSamples

    logRootPos = -1
    if (NumPosSamples == 0):
        logRootPos = 0
    else:
        logRootPos = np.log2(ProbPos)

    logRootNeg = -1
    if (NumNegSamples == 0):
        logRootNeg = 0
    else:
        logRootNeg = np.log2(ProbNeg)

    H = -((ProbPos * logRootPos) + (ProbNeg * logRootNeg))

    return [H, numSamples, NumPosSamples, NumNegSamples]


def FindMaxInformationGain(mReadableData):
    maxDict = list()

    numSamples = len(mReadableData.index)

    if (DEBUG):
        print("numsamples: " + str(numSamples))

    EntRoot = __FindEntropy(mReadableData)[0]

    for j in range(1, len(mReadableData.columns), 1):
        # right is positive

        PositiveIndexes = list()
        NegativeIndexes = list()

        for i in range(0, len(mReadableData.index), 1):
            if mReadableData.iloc[i, j] == 0:
                NegativeIndexes.append(mReadableData.index[i])
            else:
                PositiveIndexes.append(mReadableData.index[i])

        dataLeft = mReadableData.loc[NegativeIndexes]
        dataRight = mReadableData.loc[PositiveIndexes]

        EntL = __FindEntropy(dataLeft)
        NumSamplesL = EntL[1]
        NumPosSamplesL = EntL[2]
        NumNegSamplesL = EntL[3]
        EntL = EntL[0]

        EntR = __FindEntropy(dataRight)
        NumSamplesR = EntR[1]
        NumPosSamplesR = EntR[2]
        NumNegSamplesR = EntR[3]
        EntR = EntR[0]

        P_L = (NumSamplesL / numSamples)
        P_R = (NumSamplesR / numSamples)

        EntST = (P_L * EntL) + (P_R * EntR)

        Gain = EntRoot - EntST


        maxDict.append([j, data.columns[j], NumSamplesL, NumSamplesR, NumPosSamplesL, NumNegSamplesL,
                       NumPosSamplesR, NumNegSamplesR, P_L, P_R, EntST, EntST, Gain])

    maxDict.sort(key=lambda x: x[12])
    maxDict.reverse()

    print("Found Max IG: " + str(maxDict[0]) + " or " + str(maxDict[        1]))


    return maxDict


# fine the max accuracy of a machine readable dataframe
def FindMaxAccuracy(mReadableData):
    trueSub = 0
    trueIndex = 0
    tpSubList = list()

    for j in range(1, len(mReadableData.columns), 1):

        countFalsePos = 0
        countTruePos = 0
        countFalseNeg = 0
        countTrueNeg = 0

        for i in range(0, len(mReadableData.index), 1):
            c = mReadableData.iloc[i, 0]
            t = mReadableData.iloc[i, j]

            if c == 1:
                if c == t:
                    countTruePos = countTruePos + 1
                else:
                    countFalseNeg = countFalseNeg + 1
            else:
                if c != t:
                    countFalsePos = countFalsePos + 1
                else:
                    countTrueNeg = countTrueNeg + 1

        Acc = (countTruePos + countTrueNeg) / (countTruePos + countTrueNeg + countFalsePos + countFalseNeg)
        tpSubList.append([j, Acc])

    tpSubList.sort(key=lambda x: x[1])
    tpSubList.reverse()

    return tpSubList


# takes a tree node and finds its leaves
def GenerateNodes(node, idx):
    b2 = BTreeNode()
    b3 = BTreeNode()

    d = node.data

    PositiveIndexes = list()
    NegativeIndexes = list()

    for i in range(0, len(d.index), 1):
        if d.iloc[i, idx] == 0:
            NegativeIndexes.append(d.index[i])
        else:
            PositiveIndexes.append(d.index[i])

    b2.data = d.loc[NegativeIndexes]
    b2.pivot = idx
    b3.data = d.loc[PositiveIndexes]
    b3.pivot = idx

    node.left = b2
    node.right = b3

    if (DEBUG == 1):
        print("left:")
        display(b2.data)
        print("right:")
        display(b3.data)

    return node


# make a tree
def GenerateTree(mReadableData, method):
    b = BTreeNode()
    b.pivot = -1
    b.data = mReadableData

    print("\n\nRoot:")
    idx = method(b.data)

    b = BTreeNode()

    b.data = data

    b = GenerateNodes(b, idx[0][0])
    b.pivot = idx[0][0]

    print("\n\nLeft: ")
    maxL = method(b.left.data)

    if (maxL[0][0] == b.pivot):
        b.left = GenerateNodes(b.left, maxL[1][0])
        b.pivot = maxL[1][0]
        print("left Chose pivot of " + str(maxL[1]))

    else:
        b.left = GenerateNodes(b.left, maxL[0][0])
        b.left.pivot = maxL[0][0]
        print("left Chose pivot of " + str(maxL[0]))


    print("\n\nRight: ")
    maxR = method(b.right.data)
    if (maxR[0][0] == b.pivot):
        b.right = GenerateNodes(b.right, maxR[1][0])
        b.pivot = maxR[1][0]
        print("right Chose pivot of " + str(maxR[1]))
    else:
        b.right = GenerateNodes(b.right, maxR[0][0])
        b.pivot = maxR[0][0]
        print("right Chose pivot of " + str(maxR[0]))

    return b

def EvalMethod(c, whatIs : str):
    # return a tuple [TP, FP, TN, FN]
    TP = c[0]
    FP = c[1]
    TN = c[2]
    FN = c[3]

    print("this is: " + whatIs)

    confusion = dict()
    confusion["Actual Positive"] = [TP, FN]
    confusion["Actual Negative"] = [FP, TN]
    conf = pandas.DataFrame.from_dict(confusion, "index")
    conf.columns = ["Test Positive", "Test Negative"]
    print("Confusion:")
    display(conf)

    sens = TP / (TP + FN)
    spec = TN / (TN + FP)
    prec = TP / (TP + FP)
    miss = FN / (FN + TP)
    disc = FP / (FP + TP)
    omm = FN / (FN + TN)

    evals = dict()
    evals[whatIs] = [sens, spec, prec, miss, disc, omm]
    edf = pandas.DataFrame.from_dict(evals, "index")

    edf.columns = ["Sensitivity", "Specificity", "Precision", "Miss Rate",
                   "False Discovery", "False Ommision"]

    print("Evals:")
    display(edf)


# driver
if __name__ == '__main__':
    data = initialize()
    mReadableData = MakeMachineReadable(data)


    # Inspired by Ryan's implimentation.
    s = mReadableData.sample(frac=1, replace="False", random_state = RANDOMSEED)
    splits = np.array_split(s, 3)
    g1 = splits[0]
    g2 = splits[1]
    g3 = splits[2]

    t = [g1, g2]
    manufacturePivot1 = pandas.concat(t)

    t2 = [g1, g3]
    manufacturePivot2 = pandas.concat(t2)

    t3 = [g2, g3]
    manufacturePivot3 = pandas.concat(t3)

    print("TP-FP")
    TPFP1 = GenerateTree(manufacturePivot1, FindMaxTPSubFP)
    TPFP2 = GenerateTree(manufacturePivot2, FindMaxTPSubFP)
    TPFP3 = GenerateTree(manufacturePivot3, FindMaxTPSubFP)

    print("Phi")
    Phi1 = GenerateTree(manufacturePivot1, findMaxPhi)
    Phi2 = GenerateTree(manufacturePivot2, findMaxPhi)
    Phi3 = GenerateTree(manufacturePivot3, findMaxPhi)

    IG1 = GenerateTree(manufacturePivot1, findMaxPhi)
    IG2 = GenerateTree(manufacturePivot2, findMaxPhi)
    IG3 = GenerateTree(manufacturePivot3, findMaxPhi)

    IG1.PopulateMoreRoots()
    IG2.PopulateMoreRoots()
    IG3.PopulateMoreRoots()


    Phi1.PopulateMoreRoots()
    Phi2.PopulateMoreRoots()
    Phi3.PopulateMoreRoots()

    display(TPFP1.data)
    display(TPFP2.left.data)
    display(TPFP3.right.data)

    c = TPFP1.MakeConfusionWithTree(g3)
    d = TPFP2.MakeConfusionWithTree(g2)
    e = TPFP3.MakeConfusionWithTree(g1)

    EvalMethod(c, "TPFP #1")
    EvalMethod(d, "TPFP #2")
    EvalMethod(e, "TPFP #3")

    print("PHI")
    display(Phi1.data)
    display(Phi2.left.data)
    display(Phi3.right.data)

    print("Confusion Phi")
    c = Phi1.MakePhiConfusion(g3)
    d = Phi2.MakePhiConfusion(g2)
    e = Phi3.MakePhiConfusion(g1)


    EvalMethod(c, "Phi #1")
    EvalMethod(d, "Phi #2")
    EvalMethod(e, "Phi #3")

    print("IG")
    display(IG1.data)
    display(IG2.left.data)
    display(IG3.right.data)

    print("Confusion Phi")
    c = IG1.MakePhiConfusion(g3)
    d = IG2.MakePhiConfusion(g2)
    e = IG3.MakePhiConfusion(g1)


    EvalMethod(c, "IG #1")
    EvalMethod(d, "IG #2")
    EvalMethod(e, "IG #3")



csv loaded...


TypeError: 'module' object is not callable